In [1]:
import mlflow

In [2]:
MLFLOW_TRACKING_URL = "sqlite:///mlflow.db"
client = mlflow.tracking.MlflowClient(MLFLOW_TRACKING_URL)

In [3]:
client.search_experiments()

[<Experiment: artifact_location='file:///c:/Users/Booklyn/Documents/Github/mlops-zoomcamp/week-2/mlruns/2', creation_time=1716361062183, experiment_id='2', last_update_time=1716361062183, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='file:///c:/Users/Booklyn/Documents/Github/mlops-zoomcamp/week-2/mlruns/1', creation_time=1716357013132, experiment_id='1', last_update_time=1716357013132, lifecycle_stage='active', name='my-brand-new-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716355922838, experiment_id='0', last_update_time=1716355922838, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
client.search_experiments(filter_string="name='nyc-taxi-experiment'")


[<Experiment: artifact_location='file:///c:/Users/Booklyn/Documents/Github/mlops-zoomcamp/week-2/mlruns/2', creation_time=1716361062183, experiment_id='2', last_update_time=1716361062183, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [5]:
client.search_experiments(filter_string="name='nyc-taxi-experiment'")[0].name

'nyc-taxi-experiment'

In [6]:
client.get_experiment_by_name("nyc-taxi-experiment")

<Experiment: artifact_location='file:///c:/Users/Booklyn/Documents/Github/mlops-zoomcamp/week-2/mlruns/2', creation_time=1716361062183, experiment_id='2', last_update_time=1716361062183, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

## Query lowest rmse from taxi experiment

In [7]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids=2,
    filter_string="metrics.rmse < 6.5",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [8]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:4f}")

run id: f5d312fa1fb747d5915d400ed1ea657d, rmse: 6.301488
run id: 91f347f4bb994df78dc40b119a4c94c3, rmse: 6.308157
run id: bbfbbed524a04d22917dc39f03d7174a, rmse: 6.317416
run id: d8ed1104c52e41c997e73c451d0965eb, rmse: 6.318281
run id: 9ddc4d70b7a6498099bb98b6c360265b, rmse: 6.318373


## Promote

In [9]:
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URL)

In [22]:
run_id = "91f347f4bb994df78dc40b119a4c94c3"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(
    model_uri=model_uri, 
    name='nyc-taxi-regression',
    tags={"model": "test"}
                      )

Registered model 'nyc-taxi-regression' already exists. Creating a new version of this model...
Created version '6' of model 'nyc-taxi-regression'.


<ModelVersion: aliases=[], creation_timestamp=1716559745602, current_stage='None', description=None, last_updated_timestamp=1716559745602, name='nyc-taxi-regression', run_id='91f347f4bb994df78dc40b119a4c94c3', run_link=None, source='file:///c:/Users/Booklyn/Documents/Github/mlops-zoomcamp/week-2/mlruns/2/91f347f4bb994df78dc40b119a4c94c3/artifacts/model', status='READY', status_message=None, tags={'model': 'test'}, user_id=None, version=6>

In [10]:
model_name = "nyc-taxi-regression"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}, aliases: {version.aliases}")

version: 5, stage: None, aliases: []


C:\Users\Booklyn\AppData\Local\Temp\ipykernel_17388\1972632282.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [18]:
from pprint import pprint
for i in client.search_registered_models():
    pprint(dict(i), indent=4)

{   'aliases': {'champion': 1, 'dev': 2, 'production': 3, 'staging': 1},
    'creation_timestamp': 1716379403688,
    'description': 'nyc taxi',
    'last_updated_timestamp': 1716381810622,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1716381810622, current_stage='None', description=None, last_updated_timestamp=1716381810622, name='nyc-taxi-regression', run_id='91f347f4bb994df78dc40b119a4c94c3', run_link=None, source='file:///c:/Users/Booklyn/Documents/Github/mlops-zoomcamp/week-2/mlruns/2/91f347f4bb994df78dc40b119a4c94c3/artifacts/model', status='READY', status_message=None, tags={'model': 'test'}, user_id=None, version=5>],
    'name': 'nyc-taxi-regression',
    'tags': {}}


In [19]:
model_name = "nyc-taxi-regression"
for mv in client.search_model_versions(f"name='{model_name}'"):
    pprint(dict(mv), indent=4)

{   'aliases': [],
    'creation_timestamp': 1716381810622,
    'current_stage': 'None',
    'description': None,
    'last_updated_timestamp': 1716381810622,
    'name': 'nyc-taxi-regression',
    'run_id': '91f347f4bb994df78dc40b119a4c94c3',
    'run_link': None,
    'source': 'file:///c:/Users/Booklyn/Documents/Github/mlops-zoomcamp/week-2/mlruns/2/91f347f4bb994df78dc40b119a4c94c3/artifacts/model',
    'status': 'READY',
    'status_message': None,
    'tags': {'model': 'test'},
    'user_id': None,
    'version': 5}
{   'aliases': [],
    'creation_timestamp': 1716381187626,
    'current_stage': 'None',
    'description': None,
    'last_updated_timestamp': 1716381187626,
    'name': 'nyc-taxi-regression',
    'run_id': '91f347f4bb994df78dc40b119a4c94c3',
    'run_link': None,
    'source': 'file:///c:/Users/Booklyn/Documents/Github/mlops-zoomcamp/week-2/mlruns/2/91f347f4bb994df78dc40b119a4c94c3/artifacts/model',
    'status': 'READY',
    'status_message': None,
    'tags': {},
  

## Alias (stage is deprecated)

In [6]:
# create "champion" alias for version 1 of model "example-model"
client.set_registered_model_alias(model_name, "champion", 1)


In [7]:
# reassign the "Champion" alias to version 2
client.set_registered_model_alias(model_name, "Champion", 2)

In [8]:
# get a model version by alias
client.get_model_version_by_alias(model_name, "Champion")

<ModelVersion: aliases=['Champion', 'dev'], creation_timestamp=1716379521323, current_stage='None', description='', last_updated_timestamp=1716379521323, name='nyc-taxi-regression', run_id='3bae3d00dcfa422a9d66342dbef8add7', run_link='', source='file:///c:/Users/Booklyn/Documents/Github/mlops-zoomcamp/week-2/mlruns/2/3bae3d00dcfa422a9d66342dbef8add7/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'gdgboostregressor'}, user_id=None, version=2>

In [9]:
# delete the alias
client.delete_registered_model_alias(model_name, "Champion")

test inference model registry

In [20]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, alias, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{model_name}@{alias}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [21]:
df = read_dataframe("./data/green_tripdata_2023-01.parquet")

In [27]:
run_id = "3620362c7ef5440c86b0753e0ecba6ec"
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='')

'C:\\Users\\Booklyn\\Documents\\Github\\mlops-zoomcamp\\week-2\\mlruns\\2\\3620362c7ef5440c86b0753e0ecba6ec\\artifacts\\preprocessor'

In [25]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

FileNotFoundError: [Errno 2] No such file or directory: 'preprocessor/preprocessor.b'